In [2]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpcqgygbk7".


In [76]:
%%cuda
#include <stdio.h>
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <chrono>
#include <numeric> // для std::accumulate

#define BLOCK_SIZE 256

// Ядро CUDA для вычисления суммы с использованием сокращения (reduction)
__global__ void sum_reduction(const int *input, int *output, int size) {
    __shared__ int shared_data[BLOCK_SIZE];

    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    // Инициализируем shared memory (заполняем в общей памяти блока элементы)
    shared_data[tid] = (i < size) ? input[i] : 0;
    __syncthreads();

    // Сумма сокращения
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        // суть в том, что сначала мы рассчитываем значение для половины массива, потом для оставшейся половины и т.д. пока не дойдем то 1 элемента
        if (tid < stride) {
            shared_data[tid] += shared_data[tid + stride];
        }
        // синхронизация, чтобы все потоки досчитали значения
        __syncthreads();
    }

    // Сохраняем результат блока (элемент, который хранит сумму всего блока)
    if (tid == 0) {
        output[blockIdx.x] = shared_data[0];
    }
}

// Ядро CUDA для распределенного суммирования
__global__ void distributed_sum_reduction(const int *input, int *output, int size, int elements_per_thread) {
    __shared__ int shared_data[BLOCK_SIZE];

    int tid = threadIdx.x;
    int block_start_index = blockIdx.x * blockDim.x * elements_per_thread;
    int local_sum = 0;

    // Каждая нить суммирует свои элементы
    for (int i = 0; i < elements_per_thread; ++i) {
        int index = block_start_index + tid * elements_per_thread + i;
        if (index < size) {
            local_sum += input[index];
        }
    }

    // Сохраняем локальную сумму нити в shared memory
    shared_data[tid] = local_sum;
    __syncthreads();

    // Суммирование в одной нити (например, нить 0) для блока
    if (tid == 0) {
        int block_sum = 0;
        for (int i = 0; i < blockDim.x; ++i) {
            block_sum += shared_data[i];
        }
        output[blockIdx.x] = block_sum;  // Сохраняем итоговую сумму блока
    }
}

// Функция для запуска ядра CUDA (reduction)
int sum_vector_gpu(const std::vector<int>& vec) {
    int size = vec.size();
    int *d_input, *d_output;

    // Вычисляем количество блоков
    int grid_size = (size + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Выделяем память на устройстве
    cudaMalloc(&d_input, size * sizeof(int));
    cudaMalloc(&d_output, grid_size * sizeof(int));

    // Копируем данные на устройство
    cudaMemcpy(d_input, vec.data(), size * sizeof(int), cudaMemcpyHostToDevice);

    // Запускаем ядро
    sum_reduction<<<grid_size, BLOCK_SIZE>>>(d_input, d_output, size);

    // Копируем промежуточные результаты с устройства на хост
    std::vector<int> partial_sums(grid_size);
    cudaMemcpy(partial_sums.data(), d_output, grid_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Завершаем суммирование на CPU
    int total_sum = 0;
    for (int i = 0; i < grid_size; i++) {
        total_sum += partial_sums[i];
    }

    // Освобождаем память
    cudaFree(d_input);
    cudaFree(d_output);

    return total_sum;
}

// Функция для запуска распределенного ядра CUDA
int sum_vector_gpu_v2(const std::vector<int>& vec, int per) {
    int size = vec.size();
    int elements_per_thread = per;  // Количество элементов на каждую нить
    int *d_input, *d_output;

    // Вычисляем количество блоков
    int grid_size = (size + elements_per_thread * BLOCK_SIZE - 1) / (elements_per_thread * BLOCK_SIZE);

    // Выделяем память на устройстве
    cudaMalloc(&d_input, size * sizeof(int));
    cudaMalloc(&d_output, grid_size * sizeof(int));

    // Копируем данные на устройство
    cudaMemcpy(d_input, vec.data(), size * sizeof(int), cudaMemcpyHostToDevice);

    // Запускаем ядро CUDA
    distributed_sum_reduction<<<grid_size, BLOCK_SIZE>>>(d_input, d_output, size, elements_per_thread);

    // Копируем промежуточные результаты с устройства на хост
    std::vector<int> partial_sums(grid_size);
    cudaMemcpy(partial_sums.data(), d_output, grid_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Завершаем суммирование на CPU
    int total_sum = 0;
    for (int i = 0; i < grid_size; i++) {
        total_sum += partial_sums[i];
    }

    // Освобождаем память
    cudaFree(d_input);
    cudaFree(d_output);

    return total_sum;
}

// Функция для вычисления суммы на CPU
int sum_vector_cpu(const std::vector<int>& vec) {
    return std::accumulate(vec.begin(), vec.end(), 0);
}

int main() {
    // Размер вектора
    for(int i = 1000; i<1000000001; i*=10){
      const int vector_size = i; // Можно изменить на другие значения

      // Заполняем вектор значениями (например, единицами)
      std::vector<int> vec(vector_size, 1);

      // Вычисление на GPU
      auto start_gpu = std::chrono::high_resolution_clock::now();
      int sum_gpu = sum_vector_gpu(vec);
      auto end_gpu = std::chrono::high_resolution_clock::now();
      std::chrono::duration<double> elapsed_gpu = end_gpu - start_gpu;

      // Вычисление на GPU
      auto start_gpu_v2 = std::chrono::high_resolution_clock::now();
      int sum_gpu_v2 = sum_vector_gpu_v2(vec, 10000);
      auto end_gpu_v2 = std::chrono::high_resolution_clock::now();
      std::chrono::duration<double> elapsed_gpu_v2 = end_gpu_v2 - start_gpu_v2;

      // Вычисление на CPU
      auto start_cpu = std::chrono::high_resolution_clock::now();
      int sum_cpu = sum_vector_cpu(vec);
      auto end_cpu = std::chrono::high_resolution_clock::now();
      std::chrono::duration<double> elapsed_cpu = end_cpu - start_cpu;

      // Вывод результатов
      std::cout << "Сумма элементов вектора (GPU): " << sum_gpu << std::endl;
      std::cout << "Время вычисления на GPU с использованием редукции: " << elapsed_gpu.count() << " секунд" << std::endl;

      std::cout << "Сумма элементов вектора (GPU): " << sum_gpu_v2 << std::endl;
      std::cout << "Время вычисления на GPU с использованием распределнных вычислений: " << elapsed_gpu_v2.count() << " секунд" << std::endl;

      std::cout << "Сумма элементов вектора (CPU): " << sum_cpu << std::endl;
      std::cout << "Время вычисления на CPU: " << elapsed_cpu.count() << " секунд" << std::endl;
    }
    return 0;
}

Сумма элементов вектора (GPU): 1000
Время вычисления на GPU с использованием редукции: 0.0992766 секунд
Сумма элементов вектора (GPU): 1000
Время вычисления на GPU с использованием распределнных вычислений: 0.000584587 секунд
Сумма элементов вектора (CPU): 1000
Время вычисления на CPU: 1.0496e-05 секунд
Сумма элементов вектора (GPU): 10000
Время вычисления на GPU с использованием редукции: 0.000211948 секунд
Сумма элементов вектора (GPU): 10000
Время вычисления на GPU с использованием распределнных вычислений: 0.000880321 секунд
Сумма элементов вектора (CPU): 10000
Время вычисления на CPU: 0.000102448 секунд
Сумма элементов вектора (GPU): 100000
Время вычисления на GPU с использованием редукции: 0.000335278 секунд
Сумма элементов вектора (GPU): 100000
Время вычисления на GPU с использованием распределнных вычислений: 0.00109437 секунд
Сумма элементов вектора (CPU): 100000
Время вычисления на CPU: 0.00101687 секунд
Сумма элементов вектора (GPU): 1000000
Время вычисления на GPU с использ